In [20]:
import requests
from bs4 import BeautifulSoup

def obtener_links_pokemon():
    url = "https://www.wikidex.net/wiki/Lista_de_Pokémon"
    response = requests.get(url)
    soup = BeautifulSoup(response.text)


    tablas = soup.find_all('table', class_='tabpokemon')
    links_pokemon = []
    for tabla in tablas:
        filas = tabla.find_all('tr')
        for fila in filas[1:]:
            celdas = fila.find_all('td')
            if not celdas:
                continue
            if len(celdas) == 3:
                celda_nombre = celdas[0]
            elif len(celdas) > 3:
                celda_nombre = celdas[1]
            else:
                # Si la fila no cumple estas condiciones, la saltamos
                continue
            enlace_tag = celda_nombre.find('a')

            # Comprobar que la etiqueta <a> y el atributo href existan
            if enlace_tag and enlace_tag.has_attr('href'):
                enlace = enlace_tag['href']
                links_pokemon.append("https://www.wikidex.net" + enlace)

    return links_pokemon


links_pokemon = obtener_links_pokemon()[:251]
for i, link in enumerate(links_pokemon):
  print(f'{i+1}: {link}')

print(len(links_pokemon))

1: https://www.wikidex.net/wiki/Bulbasaur
2: https://www.wikidex.net/wiki/Ivysaur
3: https://www.wikidex.net/wiki/Venusaur
4: https://www.wikidex.net/wiki/Charmander
5: https://www.wikidex.net/wiki/Charmeleon
6: https://www.wikidex.net/wiki/Charizard
7: https://www.wikidex.net/wiki/Squirtle
8: https://www.wikidex.net/wiki/Wartortle
9: https://www.wikidex.net/wiki/Blastoise
10: https://www.wikidex.net/wiki/Caterpie
11: https://www.wikidex.net/wiki/Metapod
12: https://www.wikidex.net/wiki/Butterfree
13: https://www.wikidex.net/wiki/Weedle
14: https://www.wikidex.net/wiki/Kakuna
15: https://www.wikidex.net/wiki/Beedrill
16: https://www.wikidex.net/wiki/Pidgey
17: https://www.wikidex.net/wiki/Pidgeotto
18: https://www.wikidex.net/wiki/Pidgeot
19: https://www.wikidex.net/wiki/Rattata
20: https://www.wikidex.net/wiki/Rattata_de_Alola
21: https://www.wikidex.net/wiki/Raticate
22: https://www.wikidex.net/wiki/Raticate_de_Alola
23: https://www.wikidex.net/wiki/Spearow
24: https://www.wikidex.ne

In [21]:
import requests
from bs4 import BeautifulSoup

TIPOS_ES = {
    "acero","agua","bicho","dragón","eléctrico","fantasma","fuego","hada","hielo",
    "lucha","normal","planta","psíquico","roca","siniestro","tierra","veneno","volador"
}

def extraer_estadisticas_y_tipos(url):
    r = requests.get(url)
    if r.status_code != 200:
        print("Error", r.status_code, url)
        return None

    soup = BeautifulSoup(r.text, "html.parser")

    # 1) Estadísticas (PS, Ataque, Defensa, Ataque Especial, Defensa Especial, Velocidad) 
    # CAMBIO EN LOS NOMBRES DE ATAQUE ESPECIAL Y DEFENSA ESPECIAL, ANTES NO LOS COGÍA BIEN
    stats = {
        "PS": None, "Ataque": None, "Defensa": None,
        "At. esp.": None, "Def. esp.": None, "Velocidad": None
    }

    # Buscamos cualquier tabla que contenga filas TH/TD con esos nombres
    for tabla in soup.find_all("table"):
        filas = tabla.find_all("tr")
        for tr in filas:
            celdas = tr.find_all(["th","td"])
            if len(celdas) < 2:
                continue
            clave = celdas[0].get_text(" ", strip=True)
            valor = celdas[1].get_text(" ", strip=True)

            # Comprobamos si el contenido de la clave es una de las estadísticas que buscamos
            if clave in stats and (stats[clave] is None):
                # nos quedamos con el primer valor que encontremos
                # quitamos posibles símbolos y convertimos a int si se puede
                valnum = ''.join(ch for ch in valor if ch.isdigit())
                stats[clave] = int(valnum) if valnum.isdigit() else None

    # 2) Tipos (heurística por texto del enlace)  CAMBIADO, ANTES NO COGÍA BIEN TODOS LOS TIPOS
    tipos = []
    for span in soup.find_all("span", typeof="mw:File"):
        a = span.find("a")
        if a and a.has_attr("title"):
            tipo = a["title"].replace("Tipo ", "").lower()
            if tipo in TIPOS_ES and tipo not in tipos:
                tipos.append(tipo)

    # Normalizar: máximo 2 tipos
    tipos = tipos[:2]

    # 3) Nombre (del título o de la URL)
    nombre = soup.find("h1").get_text(" ", strip=True) if soup.find("h1") else url.rstrip("/").split("/")[-1]

    return {
        "nombre": nombre,
        "tipos": tipos,
        "ps": stats["PS"],
        "ataque": stats["Ataque"],
        "defensa": stats["Defensa"],
        "ataque_esp": stats["At. esp."],
        "defensa_esp": stats["Def. esp."],
        "velocidad": stats["Velocidad"],
        "url": url
    }

## Hacemos el Equipo Defensivo

In [22]:
# Equipo defensivo
import requests
from bs4 import BeautifulSoup
def obtener_pokemons_fuego():
    url = "https://www.wikidex.net/wiki/Lista_de_Pokémon"
    response = requests.get(url)
    soup = BeautifulSoup(response.text)

    pokemons_fuego = []

    tablas = soup.find_all('table', class_='tabpokemon')
    
    for tabla in tablas:
        filas = tabla.find_all('tr')
        for fila in filas[1:]:
            celdas = fila.find_all('td')
            if not celdas:
                continue

            # Pokemon de Alola. Contenido de celdas: 0:nombre, 1:tipos, 2:japonés
            if len(celdas) == 3:
                celda_nombre = celdas[0]

                celda_tipos = celdas[1]

            # Pokemon normal. Contenido de celdas: 0:número, 1:nombre, 2:tipos, 3:japonés
            elif len(celdas) > 3:
                
                celda_nombre = celdas[1]
                celda_tipos = celdas[2]     

            else:
                continue

            # Nombre y link del pokemon
            enlace_tag = celda_nombre.find('a')

            # Comprobar que la etiqueta <a> y el atributo href existan
            if enlace_tag and enlace_tag.has_attr('href'):
                enlace = enlace_tag['href']
                link_pokemon = "https://www.wikidex.net" + enlace

            if enlace_tag and enlace_tag.has_attr('title'):
                nombre = enlace_tag['title']

            # Tipos del pokemon
            tipos = []
            for span in celda_tipos.find_all("span", typeof="mw:File"):
                a = span.find("a")
                if a and a.has_attr("title"):
                    # Guardamos "Tipo Volador" como "volador"
                    tipo = a["title"].replace("Tipo ", "").lower()  
                    tipos.append(tipo)
            
            if "fuego" in tipos:
                pokemons_fuego.append({
                    "nombre": nombre,
                    "link": link_pokemon,
                    "tipos": tipos
                })

    return pokemons_fuego


In [23]:
# Visualizar lista de pokemons de tipo fuego
pokemons_fuego = obtener_pokemons_fuego()

print(f"Se han encontrado {len(pokemons_fuego)} pokémons de tipo fuego.")
i = 1
for pokemon in pokemons_fuego:
    print(f"{i} {pokemon['nombre']} ({', '.join(pokemon['tipos'])}): {pokemon['link']}")
    i += 1

Se han encontrado 85 pokémons de tipo fuego.
1 Charmander (fuego): https://www.wikidex.net/wiki/Charmander
2 Charmeleon (fuego): https://www.wikidex.net/wiki/Charmeleon
3 Charizard (fuego, volador): https://www.wikidex.net/wiki/Charizard
4 Vulpix (fuego): https://www.wikidex.net/wiki/Vulpix
5 Ninetales (fuego): https://www.wikidex.net/wiki/Ninetales
6 Growlithe (fuego): https://www.wikidex.net/wiki/Growlithe
7 Growlithe de Hisui (fuego, roca): https://www.wikidex.net/wiki/Growlithe_de_Hisui
8 Arcanine (fuego): https://www.wikidex.net/wiki/Arcanine
9 Arcanine de Hisui (fuego, roca): https://www.wikidex.net/wiki/Arcanine_de_Hisui
10 Ponyta (fuego): https://www.wikidex.net/wiki/Ponyta
11 Rapidash (fuego): https://www.wikidex.net/wiki/Rapidash
12 Marowak de Alola (fuego, fantasma): https://www.wikidex.net/wiki/Marowak_de_Alola
13 Magmar (fuego): https://www.wikidex.net/wiki/Magmar
14 Flareon (fuego): https://www.wikidex.net/wiki/Flareon
15 Moltres (fuego, volador): https://www.wikidex.net/

In [24]:
# Lista de los pokemons de tipo fuego con sus estadísticas
lista_pokemon_fuego = [extraer_estadisticas_y_tipos(pokemon['link']) for pokemon in pokemons_fuego]
print(lista_pokemon_fuego)


[{'nombre': 'Charmander', 'tipos': ['fuego', 'normal'], 'ps': 39, 'ataque': 52, 'defensa': 43, 'ataque_esp': 60, 'defensa_esp': 50, 'velocidad': 65, 'url': 'https://www.wikidex.net/wiki/Charmander'}, {'nombre': 'Charmeleon', 'tipos': ['fuego', 'normal'], 'ps': 58, 'ataque': 64, 'defensa': 58, 'ataque_esp': 80, 'defensa_esp': 65, 'velocidad': 80, 'url': 'https://www.wikidex.net/wiki/Charmeleon'}, {'nombre': 'Charizard', 'tipos': ['fuego', 'volador'], 'ps': 78, 'ataque': 84, 'defensa': 78, 'ataque_esp': 109, 'defensa_esp': 85, 'velocidad': 100, 'url': 'https://www.wikidex.net/wiki/Charizard'}, {'nombre': 'Vulpix', 'tipos': ['fuego', 'normal'], 'ps': 38, 'ataque': 41, 'defensa': 40, 'ataque_esp': 50, 'defensa_esp': 65, 'velocidad': 65, 'url': 'https://www.wikidex.net/wiki/Vulpix'}, {'nombre': 'Ninetales', 'tipos': ['fuego', 'siniestro'], 'ps': 73, 'ataque': 76, 'defensa': 75, 'ataque_esp': 81, 'defensa_esp': 100, 'velocidad': 100, 'url': 'https://www.wikidex.net/wiki/Ninetales'}, {'nombre

In [25]:
# OBTENEMOS E IMPRIMIMOS EL EQUIPO DEFENSIVO.
def obtener_equipo_defensivo(lista_pokemon_fuego):
    lista_pokemon_fuego = sorted(lista_pokemon_fuego, key=lambda p: (p['defensa'], p['ps'], p['ataque']), reverse=True)
    return lista_pokemon_fuego[:6]

def imprimir_equipo_defensivo(equipo):
    for pokemon in equipo:
        print(f"{pokemon['nombre']}\n      Defensa: {pokemon['defensa']}\n      Vida: {pokemon['ps']}\n      Ataque: {pokemon['ataque']}")
        print(f"      Ataque especial: {pokemon['ataque_esp']}\n      Defensa especial: {pokemon['defensa_esp']}\n      Velocidad: {pokemon['velocidad']}")
        print(f"      Tipos: {', '.join(pokemon['tipos'])}")

In [26]:
equipo_defensivo = obtener_equipo_defensivo(lista_pokemon_fuego)
imprimir_equipo_defensivo(equipo_defensivo)

Torkoal
      Defensa: 140
      Vida: 70
      Ataque: 85
      Ataque especial: 85
      Defensa especial: 70
      Velocidad: 20
      Tipos: fuego, veneno
Turtonator
      Defensa: 135
      Vida: 60
      Ataque: 78
      Ataque especial: 91
      Defensa especial: 85
      Velocidad: 36
      Tipos: fuego, dragón
Flamariete
      Defensa: 121
      Vida: 105
      Ataque: 115
      Ataque especial: 65
      Defensa especial: 93
      Velocidad: 91
      Tipos: fuego, dragón
Coalossal
      Defensa: 120
      Vida: 110
      Ataque: 80
      Ataque especial: 80
      Defensa especial: 90
      Velocidad: 30
      Tipos: roca, fuego
Volcanion
      Defensa: 120
      Vida: 80
      Ataque: 110
      Ataque especial: 130
      Defensa especial: 90
      Velocidad: 70
      Tipos: fuego, agua
Magcargo
      Defensa: 120
      Vida: 50
      Ataque: 50
      Ataque especial: 80
      Defensa especial: 80
      Velocidad: 30
      Tipos: fuego, roca
